In [ ]:
import pybedtools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from dotenv import load_dotenv
import scanpy as sc

load_dotenv()
DATA_PATH = Path(os.getenv("OUTPUT_PATH"))/'garcia_ATAC'


# Clustering comparison

In [ ]:
# Load cisTopic objects
import pickle
cistopic_obj_meiotic = pickle.load(open('/mnt/windows/extradata/meiotic_cells/atac_preprocessing/cistopic_obj.pkl', "rb"))

In [ ]:
cistopic_obj_garcia = pickle.load(open(DATA_PATH / 'atac_preprocessing/cistopic_obj.pkl', "rb"))

In [ ]:
len(cistopic_obj_meiotic.region_names)

In [ ]:
len(cistopic_obj_garcia.region_names)

In [ ]:
# First, create BED files with cisTopic region names
def create_bed_from_region_names(region_names):
    """Convert cisTopic region names to BED format"""
    regions = []
    for name in region_names:
        chrom, coords = name.split(':')
        start, end = coords.split('-')
        regions.append([chrom, int(start), int(end), name])
    return pybedtools.BedTool(regions)


In [ ]:

# Create BED files with correct names
meiotic_regions_bed = create_bed_from_region_names(cistopic_obj_meiotic.region_names)
garcia_regions_bed = create_bed_from_region_names(cistopic_obj_garcia.region_names)


In [ ]:
from scipy import sparse

def calculate_overlap_weights_vectorized(original_regions_bed, unified_regions_bed):
    """Vectorized version of overlap weight calculation"""
    # Get intersections as a dataframe
    overlaps_df = unified_regions_bed.intersect(original_regions_bed, wo=True).to_dataframe()
    
    # Calculate weights using pandas operations
    overlaps_df['weight'] = overlaps_df.iloc[:, -1].astype(float) / (
        overlaps_df.iloc[:, 2] - overlaps_df.iloc[:, 1]
    )
    
    # Create region names
    overlaps_df['orig_region'] = overlaps_df.iloc[:, 7]
    overlaps_df['unified_region'] = overlaps_df.iloc[:, 3]
    
    # Get ALL regions (not just those with overlaps)
    all_unified_regions = [f"{r.chrom}:{r.start}-{r.end}" for r in unified_regions_bed]
    all_original_regions = [f"{r.chrom}:{r.start}-{r.end}" for r in original_regions_bed]

    # Create unique indices for regions
    orig_to_idx = {region: idx for idx, region in enumerate(all_original_regions)}
    unified_to_idx = {region: idx for idx, region in enumerate(all_unified_regions)}
    
    # Create sparse matrix of weights
    rows = [orig_to_idx[r] for r in overlaps_df['orig_region']]
    cols = [unified_to_idx[r] for r in overlaps_df['unified_region']]
    weights = overlaps_df['weight'].values
    
    weight_matrix = sparse.csr_matrix(
        (weights, (rows, cols)),
        shape=(len(all_original_regions), len(all_unified_regions))
    )
    print(weight_matrix.max())
    return weight_matrix

In [ ]:
overlaps_df = meiotic_regions_bed.intersect(garcia_regions_bed, wo=True).to_dataframe()
overlaps_df

In [ ]:
df = meiotic_regions_bed.to_dataframe()
df.loc[df['name'] == 'chr7:20797993-20798493']

In [ ]:
overlaps_df['orig_region'] = overlaps_df.iloc[:, 3]
orig_regions = overlaps_df['orig_region'].unique()
print(len(overlaps_df), len(orig_regions))

In [ ]:
# Calculate weights using the new BED files
meiotic_weight_matrix = calculate_overlap_weights_vectorized(meiotic_regions_bed, garcia_regions_bed)

In [ ]:
meiotic_weight_matrix.min()

In [ ]:
def redistribute_counts_vectorized(matrix, weight_matrix, unified_regions):
    """Vectorized version of count redistribution"""
    print(f"Input matrix shape: {matrix.shape}")
    print(f"Weight matrix shape: {weight_matrix.shape}")    
    print(f"Number of unified regions: {len(unified_regions)}")
    
    # Ensure matrix is in CSR format for efficient operations
    matrix = sparse.csr_matrix(matrix)
    weight_matrix = sparse.csr_matrix(weight_matrix)
    
    # Process in chunks to save memory
    chunk_size = 1000
    n_chunks = (matrix.shape[1] + chunk_size - 1) // chunk_size
    result_chunks = []
    
    for i in range(n_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, matrix.shape[1])
        print(f"Processing chunk {i+1}/{n_chunks}")
        
        # Process chunk of the transposed matrix
        chunk = matrix[:, start_idx:end_idx].T
        chunk_result = chunk.dot(weight_matrix)
        result_chunks.append(chunk_result)
    
    # Vertically stack the results
    final_matrix = sparse.vstack(result_chunks)
    
    # Convert to DataFrame
    return final_matrix



In [ ]:
# Redistribute counts using vectorized approach
meiotic_unified = redistribute_counts_vectorized(
    cistopic_obj_meiotic.fragment_matrix, 
    meiotic_weight_matrix, 
    garcia_regions_bed
)


In [ ]:
import anndata as ad

# Create AnnData objects
adata_meiotic = ad.AnnData(X=meiotic_unified, 
            obs=pd.DataFrame(index=cistopic_obj_meiotic.cell_names), 
            var=pd.DataFrame(index=cistopic_obj_garcia.region_names))

adata_garcia = ad.AnnData(X=cistopic_obj_garcia.fragment_matrix.T, 
            obs=pd.DataFrame(index=cistopic_obj_garcia.cell_names), 
            var=pd.DataFrame(index=cistopic_obj_garcia.region_names))

# Add some metadata
adata_meiotic.obs['dataset'] = 'meiotic'
adata_garcia.obs['dataset'] = 'garcia'

In [ ]:
#adata_combined = ad.concat([adata_meiotic, adata_garcia], join='inner')

In [ ]:
##adata_combined

In [ ]:
#adata_combined.X.toarray().max()

In [ ]:
# import scanpy as sc
# # Process the combined data
# sc.pp.normalize_total(adata_combined)
# sc.pp.log1p(adata_combined)
# sc.pp.pca(adata_combined)
# sc.pp.neighbors(adata_combined)
# sc.tl.umap(adata_combined)

# # Plot
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
# sc.pl.umap(adata_combined, color="dataset", ax=ax1, show=False, title='Datasets')
# #sc.pl.umap(adata_combined, color="celltype", ax=ax2, show=False, title='Cell Types')
# plt.tight_layout()
# plt.show()

In [ ]:
# Also do ingest, like in celltype_inget.ipynb to get them nicely aligned.

In [ ]:
import scanpy as sc

In [ ]:
adata_garcia.obs['celltype'] = cistopic_obj_garcia.cell_data.celltype

In [ ]:
adata_garcia = adata_garcia[~adata_garcia.obs.celltype.isna()].copy()

In [ ]:
adata_garcia.shape

In [ ]:
adata_meiotic.obs['celltype'] = cistopic_obj_meiotic.cell_data.celltype

In [ ]:
adata_meiotic.shape

In [ ]:
adata_meiotic = adata_meiotic[~adata_meiotic.obs.celltype.isna()].copy()

In [ ]:
adata_meiotic.shape

In [ ]:
# Filter garcia to germ cells only
germ_celltypes = list(adata_meiotic.obs['celltype'].unique())

In [ ]:
adata_garcia = adata_garcia[adata_garcia.obs.celltype.isin(germ_celltypes)].copy()

In [ ]:
adata_garcia.shape, adata_garcia.obs['celltype'].unique()

In [ ]:
sc.pp.normalize_total(adata_garcia)
sc.pp.log1p(adata_garcia)
sc.pp.pca(adata_garcia)
sc.pp.neighbors(adata_garcia)
sc.tl.umap(adata_garcia)

In [ ]:
sc.pp.normalize_total(adata_meiotic)
sc.pp.log1p(adata_meiotic)
sc.pp.pca(adata_meiotic)
sc.pp.neighbors(adata_meiotic)
sc.tl.umap(adata_meiotic)

In [ ]:
adata_meiotic.write_h5ad(DATA_PATH / "adata_meiotic_intersection.h5ad")
adata_garcia.write_h5ad(DATA_PATH / "adata_garcia_intersection.h5ad")


In [ ]:
adata_meiotic = sc.read_h5ad(DATA_PATH / "adata_meiotic_intersection.h5ad")
adata_garcia = sc.read_h5ad(DATA_PATH / "adata_garcia_intersection.h5ad")

In [ ]:
adata_meiotic.shape, adata_garcia.shape

In [ ]:
# Ingest the new data into the reference
sc.tl.ingest(adata_meiotic, adata_garcia)

In [ ]:
from matplotlib import pyplot as plt

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
sc.pl.umap(adata_garcia, ax=ax1, color = "celltype", show=False, title='Garcia')
sc.pl.umap(adata_meiotic, ax=ax2, color = "celltype", show=False, title='Meiotic')
plt.tight_layout()
plt.show()